# GROUP PROJECT PROPOSAL

## Introduction:
**Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal**

  Classification is the process in which data is grouped into categories based on their properties. Then when presented with new data, we can use the old data to accurately predict the class of the new data. In this proposal we will state the dataset we are going to perform classification on and demonstrate preliminary data analysis while providing context as to how the process will be done and what we are expecting to find. etc.

**Clearly state the question you will try to answer with your project**

  Is there a correlation between the rank, height, age, and the hand of which a player plays? (i.e. we classify if a player is left or right handed based on the three predictors)
	
 **Identify and describe the dataset that will be used to answer the question**

The dataset we will be using is Player Stats for Top 500 Players. 


In [15]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [47]:
url = "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
download.file(url, "player_data.csv")

# player data with only the columns we want
player_data <- read_csv("player_data.csv")|> select(Age, Plays, `Current Rank`, `Best Rank`, Backhand, Height)
colnames(player_data) <- c("age", "handedness", "curr_rank", "best_rank", "backhand_grip", "height") 

# player data with all NA's removed and char fields converted to numerical and categorical fields converted to factors
player_data_clean <- player_data |>
    filter(!is.na(age), !is.na("handedness"), !is.na("curr_rank"), !is.na("best_rank"), !is.na(backhand_grip), !is.na(height)) |>
    mutate(age = as.numeric(sapply(strsplit(age," "), getElement, 1)), 
           curr_rank = as.numeric(sapply(strsplit(curr_rank," "), getElement, 1)), 
           best_rank = as.numeric(sapply(strsplit(best_rank," "), getElement, 1)),
           height = as.numeric(sapply(strsplit(height," "), getElement, 1)),
           backhand_grip = as_factor(backhand_grip),
           handedness = as_factor(handedness)
          )

player_data_clean

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,handedness,curr_rank,best_rank,backhand_grip,height
<dbl>,<fct>,<dbl>,<dbl>,<fct>,<dbl>
32,Right-handed,178,44,Two-handed,185
27,Right-handed,183,17,Two-handed,193
31,Right-handed,121,3,Two-handed,198
⋮,⋮,⋮,⋮,⋮,⋮
29,Right-handed,35,3,Two-handed,196
34,Right-handed,179,8,Two-handed,183
26,Right-handed,5,4,One-handed,185
